In [1]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize, sent_tokenize
import pandas as pd
import re
import collections
from textblob import TextBlob

In [2]:
df = pd.read_csv('/home/Downloads/dariui/articles/page_1.csv')

In [44]:
No = 66
text = df['text'][No]
cc = text.split('. ')
text_sentences = cc

company_name_alias = []
check = []

table = str.maketrans(dict.fromkeys("()"))

for m in range(len(cc)):
    words_with_parath = re.findall(r'\([^()]*\)', cc[m])
    ww = [x.translate(table) for x in words_with_parath]
    content = [x for x in ww if re.match('\\b(([A-Z]\\S*))\\b', x) is not None]
        
    if content:
        for v in range(len(cc[m].split(' '))):
            if cc[m].split(' ')[v] == "("+content[0]+")" or cc[m].split(' ')[v] == "("+content[0]+"),":
                if re.match('\\b(([A-Z]\\S*))\\b', cc[m].split(' ')[v-1]) is not None:
                    if re.match('\\b(([A-Z]\\S*))\\b', cc[m].split(' ')[v-2]) is not None:  
                        company_name_alias.append(cc[m].split(' ')[v-2])
                        company_name_alias.append(cc[m].split(' ')[v-1])
                    else:
                        company_name_alias.append(cc[m].split(' ')[v-1])
                
if len(company_name_alias) > 1:
    for n in range(1,len(company_name_alias)):
        bb = set(company_name_alias[0]).intersection(company_name_alias[n])
        check.append(bb)
        
elif len(company_name_alias)==1:
    check.append(company_name_alias)

sentences_with_tickers = []
company_name_candidates = []

for m in range(len(cc)):
    for n in range(len(company_name_alias)):
        if company_name_alias[n] in cc[m]:
                sentences_with_tickers.append(cc[m])
                
for m in range(len(sentences_with_tickers)):
    aa = set(sentences_with_tickers[0].split(' ')).intersection(sentences_with_tickers[m].split(' '))
    content = [x for x in list(aa) if re.match('\\b(([A-Z]\\S*))\\b', x) is not None]
    if content:
        company_name_candidates.append(content)
        
# Senas
company_name_alias_2 = []
check = []

for m in range(len(cc)):
    aa = set(cc[0].split(' ')).intersection(cc[m].split(' '))
    content = [x for x in list(aa) if re.match('\\b(([A-Z]\\S*))\\b', x) is not None]
    if content:
        company_name_alias_2.append(content)        

if len(company_name_alias_2) > 1:
    for n in range(1,len(company_name_alias_2)):
        bb = set(company_name_alias_2[0]).intersection(company_name_alias_2[n])
        check.append(bb)
        
elif len(company_name_alias_2)==1:
    check.append(company_name_alias_2)

company_name2 = str(min(check))[2:-2]

company_mentions = []

for m in range(len(sentences_with_tickers)):
    for n in range(len(company_name_alias)):
        if company_name_alias[n] in sentences_with_tickers[m]:
            company_mentions.append(company_name_alias[n])
            
counter=collections.Counter(company_mentions)
most_common_words = counter.most_common(5)

if most_common_words:
    most_frequent = counter.most_common(1)[0][0]

    name1 = most_common_words[0][0]+ ' ' + most_common_words[1][0]
    name2 = most_common_words[1][0]+ ' ' + most_common_words[0][0]

    if name2 in text:
        company_name = name2
    elif name1 in text:
        company_name = name1
    else:
        company_name = most_frequent
        
if company_name2 in company_name:
    company_name_final = company_name2
else:
    company_name_final = company_name
        
print("Company:", company_name_final)

text_sentences_with_company_name = []

for j in range(len(text_sentences)):
    if company_name in text_sentences[j] or 'company' in text_sentences[j]:
        text_sentences_with_company_name.append(text_sentences[j]+'. ')
        
dd = text_sentences_with_company_name[-7::]
filtered_sentences = []

for m in range(len(dd)):
    float_in = re.findall(r'[\d.]+', dd[m][:-2])
    salyga = [x for x in float_in if x == '.']
    if salyga:
        for n in range(len(dd[m].split('.'))):
            if company_name in dd[m].split('.')[n] and len(dd[m].split('.')[n])<200:
                filtered_sentences.append(dd[m].split('.')[n])
    else:
        print(m, dd[m])
        filtered_sentences.append(dd[m])

print()
print('#', No, "GIVEN SUMMARY")
summary_csv = df['summary'][No]
summary_sentences = summary_csv.split('.\n')
for j in range(len(summary_sentences)): 
    print(j+1, summary_sentences[j])

Company: Jupiter Mines
0  (Source: Jupiter Mines) The same cannot be said for most other local manganese ore miners. 
1 
 (Source: Jupiter Mines) South Africa’s position in the manganese ore sector and the effects from the lock-down  (Source: South32) As you can see from the graph above, South Africa currently accounts for around two-fifths of global manganese ore supply and is the key supplier of China. 
2  (Source: SMM) However, they are still very low compared to the highs reached in 2016 in 2018:  (Source: Jupiter Mines) Why Jupiter is well-positioned for a long lock-down The strategy of Jupiter is simple – receive dividends from Thsipi and distribute them to shareholders through dividends and share buybacks. 
3 It’s payout ratio is over 90% and the company has paid out A$0.115 ($0.08) per share in dividends since its listing in 2018. 
4  (Source: Jupiter Mines) Conclusion The coronavirus is shutting down a significant part of the global manganese ore supply at a time when China is